In [29]:
import scipy as sp
import numpy as np
import pandas as pd
import timeit
import re
import json
import pickle
import fastparquet
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.iolib.summary2 as summary2
import logging
import itertools
import math

In [31]:
# rs = fastparquet.ParquetFile("DataStore/2018-11/primclass_sim_claims_1120.parq").to_pandas()
rs = rs.dropna(subset=["sim_docvecs", "sim_claims_docvecs"])

def round_up(x, a):
    return int(math.ceil(x / a)) * a

rs["bin_sim_docvecs"] = rs["sim_docvecs"].apply(lambda x: round_up(x, 0.1))
rs["bin_sim_claims_docvecs"] = rs["sim_claims_docvecs"].apply(lambda x: round_up(x, 0.1))

In [41]:
cond = ((rs["common_pat_inv"] == False) & (rs["lawyer_match"] == False))
cols = ["common_npc_match", "inv_msa_match", "sim_docvecs", "sim_claims_docvecs", "tp_pct_common_cited"]
gr_cols = ["common_npc_match"]
rs2 = rs.loc[cond, cols].groupby(gr_cols).mean()
rs2

,inv_msa_match,sim_docvecs,sim_claims_docvecs,tp_pct_common_cited
common_npc_match,,,,
False,0.205575,0.187374,0.156023,0.001414
True,0.759259,0.321339,0.352975,0.323214


In [22]:
cond = ((rs["common_pat_inv"] == False) & (rs["lawyer_match"] == False))
cols = ["common_cited_match", "inv_msa_match", "sim_docvecs", "sim_claims_docvecs"]
gr_cols = ["common_cited_match"]
rs2 = rs.loc[cond, cols].groupby(gr_cols).mean()
rs2

,inv_msa_match,sim_docvecs,sim_claims_docvecs
common_cited_match,,,
False,0.204984,0.186501,0.155254
True,0.280979,0.269319,0.244033


In [35]:
rs.columns

Index(['tp', 'op', 'sim_claims_docvecs', 'norm_sim_claims_docvecs',
       'sim_docvecs', 'sim_ldavecs', 'tp_gyear', 'tp_naics_name',
       'op_naics_name', 'op_primclass', 'op_inv_msa', 'inv_msa_match',
       'primclass_match', 'norm_sim_ldavecs', 'norm_sim_docvecs', 'year_group',
       'common_est_inv', 'common_pat_inv', 'lawyer_match', 'num_common_cited',
       'norm_num_common_cited', 'tp_pct_common_cited',
       'norm_tp_pct_common_cited', 'common_cited_match', 'common_npc_match',
       'mean_sim_docvecs_pc', 'mean_sim_docvecs_pc_msa',
       'norm_mean_sim_docvecs_pc', 'norm_mean_sim_docvecs_pc_msa',
       'num_common_npc', 'norm_num_common_npc', 'tp_primclass_FE',
       'tp_inv_msa_FE', 'tp_examiner_FE', 'tp_lawyer_FE', 'examiner_match',
       'num_shared_abs_stemmed', 'num_shared_claims_stemmed', 'tp_primclass',
       'bin_sim_docvecs', 'bin_sim_claims_docvecs'],
      dtype='object')

In [37]:
cond = ((rs["common_pat_inv"] == False) & (rs["lawyer_match"] == False))
cols = ["tp_pct_common_cited", "inv_msa_match", "bin_sim_docvecs", "common_npc_match"]
gr_cols = ["bin_sim_docvecs"]
rs2 = rs.loc[cond, cols].groupby(gr_cols).mean()
rs2

,tp_pct_common_cited,inv_msa_match,common_npc_match
bin_sim_docvecs,,,
-0.3,0.000541,0.209003,0.000000
-0.2,0.000329,0.191571,0.000958
-0.1,0.000242,0.200959,0.000000
0.0,0.000387,0.194992,0.000000
0.1,0.000494,0.198132,0.000016
0.2,0.000852,0.201428,0.000018
0.3,0.001539,0.208259,0.000039
0.4,0.002804,0.215931,0.000051
0.5,0.004444,0.223113,0.000111


In [39]:
cond = ((rs["common_pat_inv"] == False) & (rs["lawyer_match"] == False))
cols = ["tp_pct_common_cited", "inv_msa_match", "num_shared_claims_stemmed", "common_npc_match"]
gr_cols = ["num_shared_claims_stemmed"]
rs2 = rs.loc[cond, cols].groupby(gr_cols).mean()
rs2[:20]

,tp_pct_common_cited,inv_msa_match,common_npc_match
num_shared_claims_stemmed,,,
0.0,0.000283,0.200361,0.000015
1.0,0.000511,0.205031,0.000007
2.0,0.000897,0.206009,0.000024
3.0,0.001239,0.205735,0.000025
4.0,0.001639,0.207265,0.000028
5.0,0.002275,0.205919,0.000082
6.0,0.002822,0.206549,0.000077
7.0,0.003531,0.207341,0.000071
8.0,0.004368,0.209394,0.000145


In [40]:
rs.loc[cond, cols].corr()

,tp_pct_common_cited,inv_msa_match,num_shared_claims_stemmed,common_npc_match
tp_pct_common_cited,1.000000,0.017993,0.067592,0.097351
inv_msa_match,0.017993,1.000000,0.007813,0.008682
num_shared_claims_stemmed,0.067592,0.007813,1.000000,0.010643
common_npc_match,0.097351,0.008682,0.010643,1.000000


In [27]:
# Most similar classes
sim_ps = rs[["sim_docvecs", "sim_claims_docvecs", "num_shared_claims_stemmed", "tp_primclass"]].groupby("tp_primclass").mean()
sim_ps = sim_ps.sort_values("sim_claims_docvecs", ascending=False)
sim_ps

,sim_docvecs,sim_claims_docvecs,num_shared_claims_stemmed
tp_primclass,,,
800.0,0.258423,0.408433,3.999022
544.0,0.285870,0.311219,5.023984
453.0,0.379045,0.309442,4.234568
303.0,0.253556,0.298344,5.598415
546.0,0.270566,0.296911,4.573078
301.0,0.263549,0.294771,5.368088
320.0,0.232651,0.288360,4.068665
530.0,0.225900,0.286477,2.580755
407.0,0.286995,0.279734,5.724138


In [14]:
pcn = pd.read_csv("RawData/HBSPatData/patent_class.csv")

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
pcn.head()

,Patent,Prim,Class,SubClass
0,3930270,1,360,130.24
1,3930270,0,360,84
2,3930270,0,360,271.8
3,3930270,0,G9B,15.08
4,3930269,1,360,96.3
